In [ ]:
! pip install surprise

     |████████████████████████████████| 11.8MB 6.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618257 sha256=1e8a0af6c7bfc1e92c1d5e97af05794c8f19b68148d30b692a729dc908a86e1f
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from surprise import Reader, SVD, Dataset
from surprise.model_selection.validation import cross_validate

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving ratings_small.csv to ratings_small.csv


In [ ]:
df = pd.read_csv('ratings_small.csv')

In [ ]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Split train/test

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['userId','movieId']]
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, \
                                                    random_state=1000, \
                                    stratify = df['userId'])


In [ ]:
df_train = pd.concat([X_train, y_train], axis=1)
df_train

,userId,movieId,rating
68392,475,8371,2.5
32706,236,1305,4.5
75034,520,4973,5.0
11210,73,27002,2.5
68914,477,2881,0.5
...,...,...,...
60491,439,2407,3.0
1513,15,2712,4.0
4267,23,1946,4.0
39259,286,1019,3.0


## SVD Model trained on train set

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], reader)

In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE','MAE'])

{'fit_time': (3.979448080062866,
  3.973341226577759,
  4.067822217941284,
  4.5934765338897705,
  7.250236988067627),
 'test_mae': array([0.69632289, 0.69252273, 0.69662512, 0.69265037, 0.69712699]),
 'test_rmse': array([0.90555987, 0.89924561, 0.90488878, 0.89410025, 0.90301611]),
 'test_time': (0.29870152473449707,
  0.11356759071350098,
  0.11521792411804199,
  0.20015811920166016,
  0.2005600929260254)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
df[df['userId']==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


## Evaluate on test set

In [ ]:
pred = svd.predict(1, 302, 3)

In [ ]:
pred[3]

2.6900318459273773

In [ ]:
pred

Prediction(uid=1, iid=302, r_ui=3, est=2.6900318459273773, details={'was_impossible': False})

In [ ]:
X_test

,userId,movieId
39655,290,457
79568,547,4912
8159,53,2700
14507,94,74458
36369,262,6909
...,...,...
49895,365,1784
85294,574,1097
21899,151,509
49324,358,3307


In [ ]:
y_pred = []
for line in range(len(X_test)):
  y_pred.append(svd.predict(X_test.iloc[line,0],X_test.iloc[line,1],3)[3])

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)

In [ ]:
rmse, r2

(0.9011344948501234, 0.2788542224216407)